In [23]:
!python setup.py
!python topic_sorting.py

100%|█████████████████████████████████████████| 89/89 [00:00<00:00, 2138.33it/s]


In [24]:
import pandas as pd
import numpy as np
import re
import os
from tqdm.auto import tqdm
import csv

# topics is a list of strings, each string is a topic which comes from the list of file names in the "specific_topics" folder
TOPICS = os.listdir("specific_topics") # get the list of file names in the "specific_topics" folder
TOPICS = [topic.replace(".md", "") for topic in TOPICS] # remove the ".md" from the file names
TOPICS = [topic.replace("_", " ") for topic in TOPICS] # replace the underscores with spaces


In [25]:
######## Fresh Start ########
# Starting with a csv already created and populated with prompts
# we want to distribute those prompts into markdown files that are named by the topic. To do this we need to categorize the prompts by topic.
# We can make a new column in the csv file that is the topic, and then we can use that to populate the markdown files.
# We can also use the topic column to make sure that the prompts are only added to the markdown files that are named after the topic.
# Step 1: Create a new column in the csv file that is the topic
# open the csv file
prompts_df = pd.read_csv("prompts.csv")

# Step 2: Populate the markdown files with the prompts from the csv file
prompts_df.head()

,topic,prompt,contributor,link
0,Act As A Tea-Taster,Want somebody experienced enough to distinguis...,NaN,NaN
1,Act as an Interior Decorator,I want you to act as an interior decorator. Te...,NaN,NaN
2,Act As A Florist,Calling out for assistance from knowledgeable ...,NaN,NaN
3,Act as a Self-Help Book,I want you to act as a self-help book. You wil...,NaN,NaN
4,Act as a Gnomist,I want you to act as a gnomist. You will provi...,NaN,NaN


In [26]:
df = prompts_df.copy()
df.head()

,topic,prompt,contributor,link
0,Act As A Tea-Taster,Want somebody experienced enough to distinguis...,NaN,NaN
1,Act as an Interior Decorator,I want you to act as an interior decorator. Te...,NaN,NaN
2,Act As A Florist,Calling out for assistance from knowledgeable ...,NaN,NaN
3,Act as a Self-Help Book,I want you to act as a self-help book. You wil...,NaN,NaN
4,Act as a Gnomist,I want you to act as a gnomist. You will provi...,NaN,NaN


In [27]:
df['full_text'] = df['topic'] + " " + df['prompt']
df['full_text'] = df['full_text'].str.lower()
df.head()

,topic,prompt,contributor,link,full_text
0,Act As A Tea-Taster,Want somebody experienced enough to distinguis...,NaN,NaN,act as a tea-taster want somebody experienced ...
1,Act as an Interior Decorator,I want you to act as an interior decorator. Te...,NaN,NaN,act as an interior decorator i want you to act...
2,Act As A Florist,Calling out for assistance from knowledgeable ...,NaN,NaN,act as a florist calling out for assistance fr...
3,Act as a Self-Help Book,I want you to act as a self-help book. You wil...,NaN,NaN,act as a self-help book i want you to act as a...
4,Act as a Gnomist,I want you to act as a gnomist. You will provi...,NaN,NaN,act as a gnomist i want you to act as a gnomis...


In [28]:
import pandas as pd

# extract everything after "Act as a" and assign it to the new column "category"
df['actor_name'] = df['topic'].str.extract(r'Act as[ a| ](.*)')
# remove any instances of 'a' or 'an' from the category column and replace them with an empty string in their place keeping the rest.
df['actor_name'] = df['actor_name'].str.replace(r' a | an ', ' ')
# remove double spaces
df['actor_name'] = df['actor_name'].str.replace(r'  ', ' ')

# replace any NaN values with 'General' in the category column
df['actor_name'] = df['actor_name'].fillna('General')
# make actor_name lowercase
df['actor_name'] = df['actor_name'].str.lower()
# Fill in NaN contributor values with 'None'
df['contributor'] = df['contributor'].fillna('None')

df.actor_name.value_counts()


/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_58390/3700634843.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['actor_name'] = df['actor_name'].str.replace(r' a | an ', ' ')


general                              2
a machine learning engineer          1
a legal advisor                      1
a personal chef                      1
a virtual doctor                     1
                                    ..
a lunatic                            1
an it architect                      1
an academician                       1
a developer relations consultant:    1
a life coach                         1
Name: actor_name, Length: 82, dtype: int64

In [29]:
# clear out the markdown files leaving the header line only
for topic in TOPICS:
    with open(f"specific_topics/{topic}.md", "w") as file:
        file.write(f"## Prompts for {topic}\n")



In [30]:
# scan actor_name column for words that correspond to the filenames in the "specific_topics" folder
# if a word is found, open that file and append the prompt to the file then close the file

for topic in tqdm(TOPICS, total = len(TOPICS), desc = "Writing prompts to files", colour="green"):
    for index, row in df.iterrows():
        if topic in row['actor_name']:
            with open(f"specific_topics/{topic}.md", "a") as f:
                f.write("## Prompt\n")
                write_string = "```output\n" + row['prompt'] + "\n```\n"
                f.write(write_string)
                # if there is a contributor, add the contributor to the file
                if row['contributor'] != "None":
                    f.write(f"Contributor: {row['contributor']}\n")
                f.write("\n")


Writing prompts to files: 100%|██████████| 143/143 [00:00<00:00, 482.91it/s]


In [31]:
# Now parse every word in each prompt for the words in the "industries" folder and do the same thing as above for those words in the markdown files in the "industries" folder
for industry in tqdm(os.listdir("industries"), total = len(os.listdir("industries")), desc = "Writing prompts to files", colour="green"):
    for index, row in df.iterrows():
        if industry in row['full_text']:
            with open(f"industries/{industry}.md", "a") as f:
                f.write("## Prompt\n")
                write_string = "```output\n" + row['prompt'] + "\n```\n"
                f.write(write_string)
                # if there is a contributor, add the contributor to the file
                if row['contributor'] != "None":
                    f.write(f"Contributor: {row['contributor']}\n")
                f.write("\n")

Writing prompts to files: 100%|██████████| 67/67 [00:00<00:00, 424.71it/s]


In [32]:
# now add each row of the df to markdown files in the "actors" folder based on the actor_name column where the actor_name is the name of the file
for actor in tqdm(df['actor_name'].unique(), total = len(df['actor_name'].unique()), desc = "Writing prompts to files", colour="green"):
    for index, row in df.iterrows():
        if actor in row['actor_name']:
            with open(f"actors/{actor}.md", "a") as f:
                f.write("## Prompt\n")
                write_string = "```output\n" + row['prompt'] + "\n```\n"
                f.write(write_string)
                # if there is a contributor, add the contributor to the file
                if row['contributor'] != "None":
                    f.write(f"Contributor: {row['contributor']}\n")
                f.write("\n")


Writing prompts to files: 100%|██████████| 82/82 [00:00<00:00, 421.47it/s]


In [33]:
print(TOPICS)

['swift', 'machine learning', 'cryptography', 'cloud computing', 'parallel computing', 'front end development', 'arvr', 'functional programming', 'kubernetes', 'data analysis', 'english', 'kotlin', 'big data', 'blockchain', 'desktop development', 'book', 'full stack development', 'data structures', 'mobile development', 'go', 'back end development', 'compilers', 'computer graphics', 'soa', 'python', 'monolithic', 'waterfall', 'concurrent programming', 'data science', 'pointers in computer science', 'natural language processing', 'togaf', 'networking', 'procedural programming', 'c', 'data engineering', 'event-driven programming', 'android', 'professor', 'object-oriented programming', 'generator', 'data science', 'vba', 'computer architecture', 'regex', 'iot', 'natural language processing', 'diet', 'physicist', 'scala', 'rust', 'data structures', 'deep learning', 'pmp', 'data visualization', 'parallel computing', 'cloud computing', 'excel formulas', 'creator', 'kanban', 'specific topics 

In [ ]:
# Now for every file in `specific_topics` folder, add a badge to the readme. The badge will be a link to the file.